# <center>Практическая работа 3.3. Команды Spark.</center>

<p style="text-align:right;">выполнил студент: Морозов Е.А.</p>

### Импорты, установка компонентов для работы Spark, запуск.

In [1]:
# не знаю что происходит, но так SPARK заработал в JN
# посмотрел в spark-shell пути до исполняемых файлов с пом. команд sys.path и os.environ["SPARK_HOME"]
import sys,os

sys.path.append("C:\\Users\\Morodze\\Spark\\spark-3.3.1-bin-hadoop3\\python\\lib\\py4j-0.10.9.5-src.zip")
sys.path.append("C:\\Users\\Morodze\\Spark\\spark-3.3.1-bin-hadoop3\\python")
os.environ["SPARK_HOME"] = "C:\\Users\\Morodze\\Spark\\spark-3.3.1-bin-hadoop3"

!conda activate spark
#%SPARK_HOME%\bin\pyspark - с этой строчкой вроде можно без findspark обходиться 
#                           и не делать каждый раз findspark.init() т.к. Spark будет знать домашнюю папку

!pip install py4j
import py4j

!pip install findspark
import findspark
findspark.init()

In [487]:
# запускаем SparkSession и первые импорты
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local")
    .appName("spark_pandas")
    .getOrCreate()
)

# проверка работоспособности
spark.sql('select "spark" as hello').show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [98]:
# info + UI_link
spark

SparkSession успешно создана после многочасового мозгового штурма. Попробуем использовать возможности Spark для анализа данных clickstream пользователей новостного Интернет-портала.


### Задания a, b, c.

***a.*** Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты: 

    * id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. 
    Данное поле не является первичным ключом.
    * timestamp – дата и время события в формате unix timestamp.
    * type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), переход на другую страницу(move)).
    * page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.
    * tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику
    конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.
    * sign – наличие у пользователя личного кабинета. Значения – True/False.

***b.*** Создайте датафрейм с описанной выше схемой данных.

***c.*** Наполните датафрейм данными. Пример:(12345, 1667627426, "click", 101, "Sport”, False)

Для создания данных сгенерируем шаблон с пом. Pandas. Вдобавок оказалось, что с 3-ей версии Spark поддержка методов Pandas реализована на уровне API, однако в полной мере ознакомиться с ней за отведённое время не удалось, но потенциал ощутим и это должно придать популярности кластерным вычислениям с помощью широкораспространённого инструмента. 

In [841]:
# сгенерируем данные для задания.
# импорт библиотек

import pandas as pd
import random as rnd
import numpy as np
random.seed(606)

nums = 10000 # кол-во записей

# user_id
user_id = pd.Series(rnd.choice(rnd.sample(range(1000000,1010000), (nums//10))) for i in range(nums))

# список с датами в UNIX-формате с 01-04-2022 по 01-05-2022
time = pd.Series(rnd.sample(range(1648760401,1651352401), nums))

# случайный выбор из списка событий
event_type = pd.Series([rnd.choice(['visit', 'click', 'scroll', 'move']) for i in range(nums)])

# случайный выбор из id страниц (Тип - последовательность чисел из диапазона 1000-2999)
page_id = pd.Series([rnd.choice(rnd.sample(range(1000,2999), 1000)) for i in range(nums)])

# случайный выбор из списка тэгов страницы (политика, спорт, медицина и т.д.)
page_tag = (pd.Series([rnd.choice(['politics', 'sport', 'science', 'medicine', 'football', 'restaurants', 'news',
                                   'culture', 'government', 'art', 'space', 'war', 'food', 'festivals', 'everyday'])
                       for i in range(nums)]))

# случайный выбор из вариантов наличия/отсутствия личного кабинета/подписки (сложный для анализа столбец, т.к.
# пользователи могут отказываться от подписок и заводить новые, будем считать что за месяц ситуация была неизменной)
pre_sign = pd.Series(rnd.choice([True, True, False]) for i in range(len(pandas_df['user_id'].unique())))
pre_sign_df = pd.DataFrame({'sign':pre_sign, 'user_id':pandas_df['user_id'].unique()})
sign = pandas_df.merge(pre_sign_df, how='left', left_on='user_id', right_on='user_id')['sign_y']

# собираем датасет
df = pd.DataFrame(data=pd.concat([user_id, time, event_type, page_id, page_tag, sign], axis=1))
df.columns = ['user_id', 'time', 'event_type', 'page_id', 'page_tag', 'sign']

# настройки отображения строк + shuffle
pd.set_option('display.max_rows', 100)
pandas_df = df.sample(nums).reset_index(drop=True)

display(pandas_df)
print('используем полученный датафрейм для создания датафрейма Spark.')

,user_id,time,event_type,page_id,page_tag,sign
0,1009202,1649435955,move,1415,government,True
1,1009826,1649155225,click,1800,medicine,True
2,1008250,1650239467,visit,1347,festivals,True
3,1000737,1649941576,scroll,2361,war,False
4,1009125,1650968349,click,2348,politics,True
...,...,...,...,...,...,...
9995,1006726,1650933495,visit,2472,government,False
9996,1008669,1648995292,click,1534,everyday,True
9997,1005216,1649936148,visit,2639,sport,True
9998,1002581,1649995035,visit,2310,everyday,False


используем полученный датафрейм для создания датафрейма Spark.


In [842]:
# перед созданием таблицы из pandas.dataframe сделаем импорт библиотек и создадим схему данных
import pyspark.sql.types as t
import pyspark.sql.functions as f

schema = (
    t.StructType([
        t.StructField('user_id', t.LongType(), True),
        t.StructField('time', t.LongType(), True),         # т.к. TimestampType и DateType не работают с UNIX-time
        t.StructField('event_type', t.StringType(), True),
        t.StructField('page_id', t.LongType(), True),
        t.StructField('page_tag', t.StringType(), True),
        t.StructField('sign', t.BooleanType(), True)
    ])
)

#создаём PySpark DataFrame из pandas DataFrame
df = spark.createDataFrame(pandas_df, schema = schema)
print(df)
type(df)

DataFrame[user_id: bigint, time: bigint, event_type: string, page_id: bigint, page_tag: string, sign: boolean]


pyspark.sql.dataframe.DataFrame

In [843]:
print('Фрагмент таблицы с данными в Spark и схема таблицы:')
df.show(5)
df.printSchema()

print('Резюмируем:')
df.summary().show()

Фрагмент таблицы с данными в Spark и схема таблицы:
+-------+----------+----------+-------+----------+-----+
|user_id|      time|event_type|page_id|  page_tag| sign|
+-------+----------+----------+-------+----------+-----+
|1009202|1649435955|      move|   1415|government| true|
|1009826|1649155225|     click|   1800|  medicine| true|
|1008250|1650239467|     visit|   1347| festivals| true|
|1000737|1649941576|    scroll|   2361|       war|false|
|1009125|1650968349|     click|   2348|  politics| true|
+-------+----------+----------+-------+----------+-----+
only showing top 5 rows

root
 |-- user_id: long (nullable = true)
 |-- time: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- page_id: long (nullable = true)
 |-- page_tag: string (nullable = true)
 |-- sign: boolean (nullable = true)

Резюмируем:
+-------+------------------+-----------------+----------+-----------------+--------+
|summary|           user_id|             time|event_type|          page_id|page_

In [844]:
# добавим столбец с человекочитаемой датой
df = df.withColumn('event_time', f.from_unixtime(f.col('time')))
df.show(5)

+-------+----------+----------+-------+----------+-----+-------------------+
|user_id|      time|event_type|page_id|  page_tag| sign|         event_time|
+-------+----------+----------+-------+----------+-----+-------------------+
|1009202|1649435955|      move|   1415|government| true|2022-04-08 19:39:15|
|1009826|1649155225|     click|   1800|  medicine| true|2022-04-05 13:40:25|
|1008250|1650239467|     visit|   1347| festivals| true|2022-04-18 02:51:07|
|1000737|1649941576|    scroll|   2361|       war|false|2022-04-14 16:06:16|
|1009125|1650968349|     click|   2348|  politics| true|2022-04-26 13:19:09|
+-------+----------+----------+-------+----------+-----+-------------------+
only showing top 5 rows



Первая таблица с данными об активности пользователей создана. Начнём выполнять задания.

### Задание d.

* Вывести топ-5 самых активных посетителей сайта
* Посчитать процент посетителей, у которых есть ЛК
* Вывести топ-5 страниц сайта по показателю среднего кол-ва кликов на данной странице
* Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)
* Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [845]:
# проверим работу с функциями
df.select('event_time', f.udf(lambda x: x+1000000)("page_id").alias("generated_num")).show(5)

+-------------------+-------------+
|         event_time|generated_num|
+-------------------+-------------+
|2022-04-08 19:39:15|      1001415|
|2022-04-05 13:40:25|      1001800|
|2022-04-18 02:51:07|      1001347|
|2022-04-14 16:06:16|      1002361|
|2022-04-26 13:19:09|      1002348|
+-------------------+-------------+
only showing top 5 rows



С функциями Spark работать пока не очень привычно, ниже старался решить задания как с их помощью, так и с помощью запросов на Spark-SQL API.

In [846]:
# регистрируем таблицу как временную для того чтобы драйвер видел её как SQL-таблицу
df.createOrReplaceTempView("click")

# запрос вынесем в отдельную переменную и будем использовать её как аргумент в методе
sql_expr = "SELECT COUNT(*) FROM click"
spark.sql(sql_expr).show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



* Выведем топ-5 самых активных посетителей сайта

In [847]:
# запрос вынесем в отдельную переменную и будем использовать её как аргумент в методе
sql_expr = ('''
    SELECT user_id, COUNT(event_type) as events_total
    FROM click 
    GROUP BY user_id 
    ORDER BY events_total DESC
            ''')
spark.sql(sql_expr).show(5)

+-------+------------+
|user_id|events_total|
+-------+------------+
|1008461|           8|
|1003051|           7|
|1000391|           6|
|1001022|           6|
|1005634|           6|
+-------+------------+
only showing top 5 rows



In [848]:
# методами Spark (на этом этапе я ещё не понимал как присвоить alias нужной колонке)
df.groupBy('user_id').agg({'event_type':'count'}).sort('count(event_type)', ascending=False).limit(5).show()

+-------+-----------------+
|user_id|count(event_type)|
+-------+-----------------+
|1008461|                8|
|1003051|                7|
|1001361|                6|
|1000391|                6|
|1002359|                6|
+-------+-----------------+



* Посчитаем процент посетителей, у которых есть ЛК

In [849]:
sql_expr = ('''
    SELECT ROUND(100*(COUNT(DISTINCT(user_id))/(
                                                SELECT COUNT(DISTINCT(user_id)) FROM click
                                                )), 2) AS perc_reg_users
    FROM click
    WHERE sign="true"
            ''')
spark.sql(sql_expr).show()

+--------------+
|perc_reg_users|
+--------------+
|         75.68|
+--------------+



* Вывести топ-5 страниц сайта по показателю среднего кол-ва кликов на данной странице

Задачу смог трактовать только как сравнение страниц, на которые приходились клики по количеству этих самых пришедшихся кликов. Средняя величина в текущем датасете получается слишком небольшой из-за сравнимого кол-ва страниц и пользователей.

In [850]:
sql_expr = ('''
    SELECT page_id, COUNT(event_type) AS clicks_count
    FROM click
    WHERE event_type="click"
    GROUP BY page_id
    ORDER BY COUNT(event_type) DESC
    LIMIT 5
            ''')
spark.sql(sql_expr).show(5)

+-------+------------+
|page_id|clicks_count|
+-------+------------+
|   2923|           6|
|   1090|           6|
|   1963|           6|
|   1433|           6|
|   1801|           6|
+-------+------------+



* Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [852]:
sql_expr = ('''
    SELECT *,
           CASE
               WHEN EXTRACT(HOUR from event_time) BETWEEN 0 AND 3 THEN '0-4'
               WHEN EXTRACT(HOUR from event_time) BETWEEN 4 AND 7 THEN '4-8'
               WHEN EXTRACT(HOUR from event_time) BETWEEN 8 AND 11 THEN '8-12'
               WHEN EXTRACT(HOUR from event_time) BETWEEN 12 AND 15 THEN '12-16'
               WHEN EXTRACT(HOUR from event_time) BETWEEN 16 AND 19 THEN '16-20'
               ELSE '20-24'
           END AS period
    FROM click
            ''')
spark.sql(sql_expr).createOrReplaceTempView("click_1")
df.show(3)

+-------+----------+----------+-------+----------+----+-------------------+
|user_id|      time|event_type|page_id|  page_tag|sign|         event_time|
+-------+----------+----------+-------+----------+----+-------------------+
|1009202|1649435955|      move|   1415|government|true|2022-04-08 19:39:15|
|1009826|1649155225|     click|   1800|  medicine|true|2022-04-05 13:40:25|
|1008250|1650239467|     visit|   1347| festivals|true|2022-04-18 02:51:07|
+-------+----------+----------+-------+----------+----+-------------------+
only showing top 3 rows



* Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [853]:
sql_expr = ('''
    SELECT period,
           COUNT(event_type) AS events_count
    FROM click_1
    GROUP BY period
            ''')
spark.sql(sql_expr).sort(f.desc('events_count')).limit(1).show()

+------+------------+
|period|events_count|
+------+------------+
| 20-24|        1721|
+------+------------+



Первая часть задания выполнена, обратимся ко второй, создадим необходимую таблицу.

### Задание d.

Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов:
                        
    * Id – уникальный идентификатор личного кабинета
    * User_id – уникальный идентификатор посетителя
    * ФИО посетителя
    * Дата создания ЛК

Затем необходимо
* Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
* Выведите 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.
* Вывести топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

In [320]:
# для создания 2 таблицы также сгенерируем данные 

# установка и импорты библиотеки для генерации имён
pip install mimesis

     ---------------------------------------- 4.4/4.4 MB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [837]:
from mimesis import Person
from mimesis.locales import Locale
from mimesis.enums import Gender
person = Person(Locale.EN)

# генерируем список имён и добавляем к нему уникальные индексы из таблицы 1 (есть повторяющиеся user_id)
# Поскольку в дальнейших заданиях пол пользователя играет роль, добавим его в список к генерируемому имени
f_name = pd.Series((rnd.choice([[person.full_name(gender=Gender.FEMALE),'FEMALE'],[person.full_name(gender=Gender.MALE),'MALE']])
                   for i in range(len(pandas_df['user_id'].unique()))), name='fn_g')

f_name = f_name.to_frame()
f_name['name'] = f_name['fn_g'].apply(lambda x: x[0])
f_name['gender'] = f_name['fn_g'].apply(lambda x: x[1])

name_df = pd.DataFrame({'full_name':f_name['name'], 'gender':f_name['gender'], 'user_id':pandas_df['user_id'].unique()})

# добавляем список id личного кабинета пользователей
name_df['user_acc'] = name_df['user_id']*2+111

# список с датами в UNIX-формате с 01-01-2000 по 31-11-2022 для создания столбца с датами регистрации ЛК
name_df['acc_reg'] = pd.Series(rnd.sample(range(946674001,1667163601), len(pandas_df['user_id'].unique())))
name_df.sample(5)

,full_name,gender,user_id,user_acc,acc_reg
5485,Avelina Mayo,FEMALE,1002331,2004773,1498947873
4942,Laurice Martinez,FEMALE,1007374,2014859,1317670664
3427,Mark Dunlap,MALE,1003454,2007019,1048994625
758,Carl Hopkins,MALE,1007197,2014505,1006086679
5085,Margarito Calderon,MALE,1000653,2001417,1646376177


Таблица создана, очевидно её судьба - быть отправленной в Spark и объединиться с созданной ранее pandas_df для выполнения последующих заданий. Предпримем соответствующие шаги ниже.

In [854]:
# снова прежде создания таблицы из pandas.dataframe создадим схему данных

schema = (
    t.StructType([
        t.StructField('full_name', t.StringType(), True),
        t.StructField('gender', t.StringType(), True),         # т.к. TimestampType и DateType не работают с UNIX-time
        t.StructField('user_id', t.LongType(), True),
        t.StructField('user_acc', t.LongType(), True),
        t.StructField('acc_reg', t.LongType(), True)
    ])
)

#создаём PySpark DataFrame из pandas DataFrame
df_n = spark.createDataFrame(name_df, schema = schema)
print(df)
type(df)

DataFrame[user_id: bigint, time: bigint, event_type: string, page_id: bigint, page_tag: string, sign: boolean, event_time: string]


pyspark.sql.dataframe.DataFrame

In [859]:
# добавим столбец с человекочитаемой датой
df_n = df_n.withColumn('acc_registred', f.from_unixtime(f.col('acc_reg')))
df_n.show(3)

+----------------+------+-------+--------+----------+-------------------+
|       full_name|gender|user_id|user_acc|   acc_reg|      acc_registred|
+----------------+------+-------+--------+----------+-------------------+
|       Kyle Boyd|  MALE|1004985| 2010081| 976382380|2000-12-09 20:19:40|
|Christian Hooper|  MALE|1007864| 2015839|1466459654|2016-06-21 00:54:14|
|    Stuart Chase|  MALE|1004287| 2008685|1173306494|2007-03-08 01:28:14|
+----------------+------+-------+--------+----------+-------------------+
only showing top 3 rows



* Выведем фамилии посетителей, которые читали хотя бы одну новость про спорт.

Сперва объединим таблицы и затем отберём нужные записи.

In [910]:
# объединим таблицы по общему полю и найдём их пересечение

join_expression = df['user_id'] == df_n['user_id']
df_full = (
          df.join(df_n, join_expression, 'left')
            .select(df['user_id'], 'full_name', 'gender', 'user_acc', 'acc_registred',
                    'event_time', 'event_type', 'page_id', 'page_tag', 'sign')
)
q = df_full.filter('page_tag like "%sport%" and event_type like "%visit%"' ).select('full_name').count()
print(f'Новости спорта просмотрело {q} посетителя, например:')

df_full.filter('page_tag like "%sport%" and event_type like "%visit%"' ).select('full_name').show(10)

Новости спорта просмотрело 173 посетителя, например:
+----------------+
|       full_name|
+----------------+
|Coretta Davidson|
|       Anja Cruz|
|  Dorsey Osborne|
|     Stacey Ford|
|  Francina Duran|
| Taneka Erickson|
|  Arnold Lindsay|
|      Galen Hunt|
|     Keira Bauer|
|   Teddy William|
+----------------+
only showing top 10 rows



* Выведем 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.

Отмечу, что в изначально сгенерированном датасете хранится список активностей на сайте за условный 1 месяц, а даты регистрации отстоят от него порою на 20 лет в прошлое, это также принятая условность. 

In [967]:
# в переменной сохраним сгрупированный объект (по уник. user_id со столбцом с макс. разницей во времени посещения)
df_full_wdiff = (df_full
    .withColumn('diff_days', f.datediff(f.col('event_time'),f.col('acc_registred')))
    .groupBy('user_id', 'full_name')
    .agg({'diff_days':'max'})
    .orderBy(f.desc('max(diff_days)'))
    .toDF('user_id', 'full_name', 'diff_days')
    .select('user_id', 'full_name', 'diff_days')
)

# для показа 10% найдём длину списка уникальных user_id используя метод .count() к переменной df_full_wdiff
df_full_wdiff.show(df_full_wdiff.distinct().count()//10)

+-------+--------------------+---------+
|user_id|           full_name|diff_days|
+-------+--------------------+---------+
|1007688|        Kimber Tyson|     8148|
|1004579|           Gary Moon|     8147|
|1006298|            Rea Kidd|     8145|
|1009906|        Ariane Tyson|     8144|
|1005979|     Chassidy Farley|     8142|
|1001208|        Brett Bailey|     8138|
|1004915|     Lawrence Prince|     8138|
|1004335|        Sage Navarro|     8137|
|1007839|    Christinia Rojas|     8136|
|1002251|   Eleonore Sheppard|     8136|
|1006583|       Nelia Skinner|     8131|
|1006735|         Leia Willis|     8128|
|1003659|        Tilda Albert|     8128|
|1009897|      Robert Edwards|     8126|
|1001725|        Reggie Blair|     8126|
|1002940|   Pearly Cunningham|     8125|
|1000358|       Palmira Beach|     8124|
|1006615|        Kevin Barlow|     8124|
|1007113|      Virgen Lindsey|     8123|
|1002470|           Drew Cote|     8123|
|1000298|        Jess Padilla|     8122|
|1005108|       

* Выведем топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

In [1000]:
(
df_full
    .filter('event_type like "visit" or event_type like "move"')
    .groupBy('gender', 'page_id')
    .agg({'page_id':'count'})
    .toDF('gender', 'top-5_pages', 'total_views')
    .orderBy(f.desc('total_views'))
    .limit(12)
    .orderBy('gender', f.desc('total_views'))
    .show(10)

)

+------+-----------+-----------+
|gender|top-5_pages|total_views|
+------+-----------+-----------+
|FEMALE|       1893|          7|
|FEMALE|       2027|          6|
|FEMALE|       1849|          6|
|FEMALE|       2421|          6|
|FEMALE|       1687|          6|
|  MALE|       2625|          7|
|  MALE|       2854|          7|
|  MALE|       1299|          7|
|  MALE|       2773|          6|
|  MALE|       2358|          6|
+------+-----------+-----------+
only showing top 10 rows



(задание **e** для пытливых умов пропущено, времени на его освоение не слишком много, задача довольно интересна поставлена, ведь наверняка Spark имеет интерфейсы для подключения к БД и работы с драйвером БД)

### Задание f. 

* Редакция совместно с аналитиками хотят провести масштабную рекламную кампанию, в рамках которой на сайте будут выпущены 3 новости с новой тематикой. Рекламный бюджет позволяет охватить только 10% доступной вам аудитории посетителей сайта. Напишите запрос, который позволит вычислить целевую аудиторию для данных новостей. Постарайтесь объяснить ваше решение.

Банальная логика подсказывает, что если тематика планируемых новостей в каком-то смысле родственна имеющимся на сайте рубрикам, то в данном случае хорошо бы найти пользователей, обладающих схожими векторами интересов, они предположительно будут похожим образом воспринимать и новую информацию. Вектора интересов сравнивать по принятой мере расстояния (скорее всего косинусной). Если новости планируются "из ряда вон" и никаких похожих тематик на сайте ранее не существовало, то стоит набрать случайную выборку пользователей с разнообразными векторами интересов. Такое исследование требует большего времени, чем можно себе позволить в этой задаче. Ниже пользователи отобраны более простыми инструментами, имеющимися под рукой, и исходя из более простых предположений, что в силу Центральной Предельной Теоремы скорее всего даст схожие результаты из-за случайности в распределении величин.

In [1016]:
# выведем список самых популярных новостных категорий среди страниц, которые просматривались пользователями
(
    df_full
    .filter('event_type like "%visit%"')
    .groupBy('page_tag')
    .agg({'event_type':'count'})
    .orderBy(f.desc('count(event_type)'))
    .show()
)

+-----------+-----------------+
|   page_tag|count(event_type)|
+-----------+-----------------+
|   everyday|              186|
| government|              183|
|restaurants|              178|
|    culture|              175|
|       food|              175|
|        war|              174|
|      sport|              173|
|  festivals|              172|
|    science|              169|
|      space|              167|
|   politics|              166|
|        art|              162|
|   medicine|              160|
|       news|              157|
|   football|              148|
+-----------+-----------------+



Можно составить представвление о наиболее популярных тематиках, среди тех статей, что были прочитаны, что даёт определённую пищу для размышления при подготовке новых статей и отборе пользователей для рассылки.

In [1020]:
# выведем список самых читающих пользователей, желательно также учитывать давность посещений, но поскольку у нас
# имеется список активности за месяц, то можно считать этот довод учтённым.
(
    df_full
    .filter('event_type like "%visit%"')
    .groupBy('user_id')
    .agg({'event_type':'count'})
    .orderBy(f.desc('count(event_type)'))
    .show()
)

+-------+-----------------+
|user_id|count(event_type)|
+-------+-----------------+
|1001361|                4|
|1004749|                3|
|1008913|                3|
|1003508|                3|
|1007152|                3|
|1008606|                3|
|1009684|                3|
|1009493|                3|
|1006578|                3|
|1009559|                3|
|1003408|                3|
|1005139|                3|
|1007933|                3|
|1005334|                3|
|1001156|                3|
|1002925|                3|
|1009411|                3|
|1006531|                3|
|1007085|                3|
|1003699|                3|
+-------+-----------------+
only showing top 20 rows



In [1023]:
# выведем список самых читающих пользователей, желательно также учитывать давность посещений, но поскольку у нас
# имеется список активности за месяц, то можно считать этот довод учтённым.
(
    df_full
    .filter('event_type like "%visit%"')
    .groupBy('user_id')
    .agg({'event_type':'count'})
    .orderBy(f.desc('count(event_type)'))
    .show()
)

+-------+-----------------+
|user_id|count(event_type)|
+-------+-----------------+
|1001361|                4|
|1004749|                3|
|1008913|                3|
|1003508|                3|
|1007152|                3|
|1008606|                3|
|1009684|                3|
|1009493|                3|
|1006578|                3|
|1009559|                3|
|1003408|                3|
|1005139|                3|
|1007933|                3|
|1005334|                3|
|1001156|                3|
|1002925|                3|
|1009411|                3|
|1006531|                3|
|1007085|                3|
|1003699|                3|
+-------+-----------------+
only showing top 20 rows



Видно что только примерно 300 пользлователей читало за месяц более 1 статьи, всех активных пользователей разумно добавить в рассылку. Для окончательного выбора аккаунтов можно несколько расширить условие отбора записей оператором "ИЛИ", вдобавок из-за особенностей сгенерированной выборки наиболее популярные категории попадут в этот список сами собой и дополнительной фильтрации не потребуется. Остановимся на достигнутом результате, на более глубокое исследование времени явно не достаточно.

In [1028]:
# выведем список пользователей для рассылки

(
    df_full
    .filter('event_type like "%visit%" OR event_type like "%move%"')
    .groupBy('user_id')
    .agg({'event_type':'count'})
    .orderBy(f.desc('count(event_type)'))
    .show(df_full_wdiff.distinct().count()//10)
)

+-------+-----------------+
|user_id|count(event_type)|
+-------+-----------------+
|1003133|                5|
|1008461|                5|
|1005635|                4|
|1002359|                4|
|1008486|                4|
|1000391|                4|
|1003684|                4|
|1004731|                4|
|1005634|                4|
|1003658|                4|
|1006816|                4|
|1005903|                4|
|1001361|                4|
|1006549|                4|
|1003699|                4|
|1000227|                4|
|1001022|                4|
|1001034|                4|
|1005621|                3|
|1008894|                3|
|1009702|                3|
|1007680|                3|
|1007896|                3|
|1003269|                3|
|1008996|                3|
|1002128|                3|
|1008548|                3|
|1000873|                3|
|1008592|                3|
|1003325|                3|
|1000184|                3|
|1002024|                3|
|1004473|           

In [ ]:
# отсанавливаем процесс в самом конце
spark.stop()